<a href="https://colab.research.google.com/github/luanSaid/caseIntelbrasDataScience/blob/main/case_intelbras_luan_said.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case - Intelbras

Teste técnico desenvolvido por **Luan Said Meira** para fins avaliativos no processo seletivo para a vaga de Analista Júnior no Squade de Ciência de Dados na Intelbras.

# Configuração do ambiente

**Para executar o código corretamente, modifique o valor das variáveis abaixo de acordo com o local onde se encontram os arquivos no seu computador**

In [3]:
INPUT_URL = '/content/drive/MyDrive/Faculdade/Estágios/Processos seletivos/Intelbras/arquivo-desafio.csv'

# Data Wrangling

In [5]:
import pandas as pd
from datetime import datetime

In [6]:
df = pd.read_csv(INPUT_URL, sep=';')
df.columns = ['nome_funcionario', 'id_cargo', 'cargo', 'categoria', 'nome_unidade', 'jornada', 'situacao', 'data_admissao', 'data_desligamento'] # Rename columns

In [ ]:
df = df.fillna(0)                       # Preenche com 0 os campos Nan (Not a Number)
view = pd.DataFrame()                   # Criação de dataframe para 
df = df.replace('\s+', ' ', regex=True) # Remove os espaços excedentes
df.head(10)

# Análise e exploração dos dados

**Funcionários que possuem o sobrenome 'Silva'**

In [ ]:
view = pd.DataFrame()
view['nome_funcionario'] = df['nome_funcionario'].str.lower()                   # Transformação do campo para minúsculo
view['contains_silva'] = view.nome_funcionario.str.contains(' silva')           # Verificação da ocorrência da str  
qtd_silva = view[view['contains_silva'] == True].count()

print('---------------------------------------------------------------------')
print(f'Existem {qtd_silva[1]} funcionários que possuem o sobrenome "Silva", sendo esses:')
print('---------------------------------------------------------------------\n')
print(view['nome_funcionario'][view['contains_silva'] == True])


**Funcionárias que possuem o primeiro nome "Ana"**

In [ ]:
view = pd.DataFrame()
view['nome_funcionario'] = df['nome_funcionario'].str.lower()                   # Transformação do campo para minúsculo
view['start_with_ana'] = df.nome_funcionario.str.startswith('Ana ')             # Verificação da ocorrência da str no início
qtd_ana = (view[view['start_with_ana'] == True]).count()

print('----------------------------------------------------------------------')
print(f'Existem {qtd_ana[1]} funcionárias que possuem o primeiro nome "Ana", sendo elas:')
print('----------------------------------------------------------------------\n')
print(view['nome_funcionario'][view['start_with_ana'] == True])


**Tipos de cargos existentes no dataset**

In [ ]:
view = pd.DataFrame()
view['cargos_unicos'] = df['cargo'].unique()
qtd_cargo = view['cargos_unicos'].count()

print('-------------------------------------------------')
print(f'Existem {qtd_cargo} cargos distintos na base, sendo esses:')
print('-------------------------------------------------\n')

print(view['cargos_unicos'].sort_values(ascending=True))

**Percentual de funcionários que possuem a carga igual a 40 horas por semana**

In [ ]:
jornada = df.groupby(['jornada']).size()
percentual = int(jornada['40 horas semanais']) * 100 / df.shape[0]

print('------------------------------------------------------------------------------')
print(f'{percentual:.2f}% dos funcionários possuem carga de trabalho igual a 40 horas por semana.')
print('------------------------------------------------------------------------------')

**Nome do funcionário que possui a data de admissão em "15/06/2012" e data de desligamento em "04/12/2012"**

*Observação: O registro retornado dessa consulta não possui categoria, ou seja, logo adiante ele será removido, de acordo com os exercícios propostos.*

In [ ]:
view = pd.DataFrame() 
view = df.copy()                                                                # Cópia do dataframe original

view.data_admissao = pd.to_datetime(view.data_admissao)                         # Conversão dos campos em datetime
view.data_desligamento = pd.to_datetime(view.data_desligamento)

view.data_admissao = view.data_admissao.dt.strftime('%d/%m/%Y')
view.data_desligamento = view.data_desligamento.dt.strftime('%d/%m/%Y')         # Formatação de acordo com o padrão 'dia/mês/ano'

search = view[(view['data_admissao'] == '15/06/2012') & (view['data_desligamento'] == '04/12/2012')]

print(search['nome_funcionario'])


**Criação da coluna chamada "nome_situacao" com a concatenação do campo "nome_funcionario" e campo "situacao"**

In [ ]:
df['nome_situacao'] = df['nome_funcionario'] + ' - ' + df['situacao']
df.head(10)

**Remoção de todos os registros que não possuam categoria**

In [ ]:
view = pd.DataFrame()                                                           # Criação de cópia para possível busca recuperação de dados posterior
view = df.copy()

view = view[view.categoria != 0]
view.head(5)

**Profissionais ativos que podem se aposentar com base apenas no cargo atual**

Processo ilustrativo, no qual avalia somente o tempo de contribuição era superior a 30 anos (média entre os valores de contribuição).

In [ ]:
search = df.copy()
now = datetime.now()

search = search[search.situacao == 'Ativo']
search['ano_admissao'] = search.data_admissao.str.slice(0,4,1)
search['tempo_contribuicao'] = (now.year - search.ano_admissao.astype(int))


result = search[(search['tempo_contribuicao'].astype(int) >= 30)]
quantidade = result['nome_funcionario'].count()

print(f'{quantidade} funcionários já podem se aposentar ao longo desse ano de {now.year}, abaixo seguem seus dados:')
result[['nome_funcionario', 'cargo', 'data_admissao', 'tempo_contribuicao']].sort_values(by='tempo_contribuicao', ascending=False)

**Download da nova planilha tratada**

Observação: Substituir o endereço abaixo para um local de armazenamento de sua preferência

In [ ]:
view.to_csv(r'/content/drive/MyDrive/Faculdade/Estágios/Processos seletivos/Intelbras/exported_file_data.csv', header=True)